<h1> <center> Titanic Hands on Example</h1> </center>
 <img width="150"  align="right" src="tbyds.jpg">
The steps:
1. Asking the right question
2. Identifying and preparing the right data.
3. Select a ML Algorithm and apply the algorithm to get a candidate model.  (A.K.A. Training the model).
4. Evaluate our results and adjust where needed.  
5. Present the model and reiterate (good models only get better).  



Main Kaggle Titanic Page: https://www.kaggle.com/c/titanic

## 1. Asking the right question
### 1. 1 Trying to predict survived.
- What kind of ML problem is this?
- Women and children?  (Gender and Age)
- How did class play out?


## 2. Identifying and preparing the right data.
### 2.1 - Reviewing the data we have (general review)

In [ ]:
#IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import various algorithms for later
from sklearn import linear_model, preprocessing, tree, model_selection, svm
from sklearn.naive_bayes import GaussianNB

#READ DATA FRAME
train_df=pd.read_csv("./train.csv")

#REVIEW DATA
train_df.head()

In [ ]:
#REVIEW DATA FORMAT 

train_df.info() 

### Observe!
- Notice null values - 891 entries, but only:
    - 714 age
    - 204 cabin 
    - 889 embarked  
- Where are there mismatched types?
    - Gender?
    
- Feature selection
    - Ask yourself- what features do we need?
    - Embarked?
    - Cabin?
    - What's relevant?

In [ ]:
#Distribution of survival 
print(train_df.Survived.value_counts())

train_df.Survived.value_counts(normalize=True).plot(kind="bar")
plt.title("Distribution of Survival (1=Survived)")
plt.show()

In [ ]:
#Distribution of passenger gender 
print(train_df.Sex.value_counts())
train_df.Sex.value_counts().plot(kind="bar")
plt.title("Distribution of Passenger Gender")
plt.show()

In [ ]:
#Age Histogram
train_df["Age"].hist(bins=100) 
plt.title("Age Histogram")
plt.show()

In [ ]:
#Distribution of passengers by class
print(train_df.Pclass.value_counts())
train_df.Pclass.value_counts().plot(kind="bar")
plt.title("Distribution in passenger classes")
plt.show()

In [ ]:
#Class in respect to age
for x in [1,2,3]:
    train_df.Age[train_df.Pclass==x].plot(kind="kde")
    
plt.title("Age Distribution in classes")
plt.legend(("First Class","Second Class","Third Class"))
plt.show()


In [ ]:
#Embarked locations
print(train_df.Embarked.value_counts())
train_df.Embarked.value_counts().plot(kind='bar')
plt.title("Passengers by Embarked Location")
plt.show()

## 2. Identifying and preparing the right data.
### 2.2 - Reviewing the data we have (in regards to survival rate)

In [ ]:
#Age in regards to survival rate
plt.scatter(train_df.Survived,train_df.Age,alpha=0.05)
plt.title("Age in regards to Survival Rate (1 = Survived)")
plt.show()

In [ ]:
#Gender of survived 
print(train_df.Sex[train_df.Survived == 1].value_counts())
train_df.Sex[train_df.Survived == 1].value_counts(normalize=True).plot(kind="bar")
plt.title("Gender distribution of those that survived")
plt.show()

In [ ]:
#Survival rate by gender 
print(train_df.groupby('Sex').Survived.mean())
train_df.groupby('Sex').Survived.mean().plot(kind="bar")
plt.title("Distribution of survival rate by gender")
plt.show()

In [ ]:
#Class in respect to survived
for x in [1,2,3]:
    train_df.Survived[train_df.Pclass==x].plot(kind="kde")
    
plt.title("Survived in respect to class")
plt.legend(("1","2","3"))
plt.show()

In [ ]:
print(train_df.groupby(['Pclass', 'Sex']).Survived.mean())
train_df.groupby(['Pclass', 'Sex']).Survived.mean().plot(kind="bar")
plt.title("Survived in respect to class")
plt.show()

## 2. Identifying and preparing the right data.
### 2.3 - Remove the features we don't need.

In [ ]:
features =  ['PassengerId','Survived','Pclass','Sex','Age','SibSp','Parch','Fare']
train_df = train_df[features]
train_df.head()

#note that - we could also use features_v = train_df.drop(['Name','Ticket','Cabin','Embarked'],axis=1)

## 2. Identifying and preparing the right data.
### 2.4 - Correct data types - Gender

In [ ]:
# Convert the male and female groups to integer form
train_df.Sex = train_df.Sex.map({"male": 0, "female":1})
train_df.info()
train_df.head()

#note - factorize could be helpful here, but causes an ugly error message 
#"A value is trying to be set on a copy of a slice from a DataFrame."
#turn off warning http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

#code:
#pd.options.mode.chained_assignment = None  # default='warn' 
#train_df['Sex']=train_df.factorize(train_df['Sex'])[0]

## 2. Identifying and preparing the right data.
### 2.5 - Fill in average age


In [ ]:
#Naive way to do this below
#train_df=train_df.dropna()
#train_df.info()

#A basic mean would look as follows:
train_df["Age"]=train_df["Age"].fillna(train_df["Age"].dropna().median())
train_df.info()



## 3. Select a ML Algorithm and apply the algorithm to get a candidate model.  (A.K.A. Training the model).
### 3.1 Setup

In [ ]:
#simple model (If you're a woman, I'll guess you'll survive!)

train_df["Hypothesis"]=0 
train_df.loc[train_df.Sex == 1,"Hypothesis"]=1 

train_df["Result"]=0
train_df.loc[train_df.Survived == train_df["Hypothesis"],"Result"]=1
             
print(train_df["Result"].value_counts(normalize=True))

In [ ]:
#Regression

In [ ]:
#Setting up for models
training_feature_names=["Pclass","Age","Fare","Sex","SibSp","Parch"]
training_target=train_df["Survived"].values  #array of survived 
training_features=train_df[training_feature_names].values  #array of the features 

In [ ]:
MLalgorithm = linear_model.LogisticRegression(solver="liblinear")

#Linear Model
MLmodel  = MLalgorithm.fit(training_features,training_target)
print ("Linear model score:", MLmodel.score(training_features,training_target))

scores = model_selection.cross_val_score(MLalgorithm,training_features,training_target,scoring='accuracy',cv=50)
print(scores.mean())

#Polynomial Model
poly = preprocessing.PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(training_features)
MLmodel  = MLalgorithm.fit(poly_features,training_target)
print ("Polynomial model score:", MLmodel.score(poly_features,training_target))

scores = model_selection.cross_val_score(MLalgorithm,poly_features,training_target,scoring='accuracy',cv=50)
print(scores.mean())

In [ ]:
#Trees (overfit)

In [ ]:
MLalgorithm = tree.DecisionTreeClassifier(random_state=1)
MLmodel = MLalgorithm.fit(training_features,training_target)
print("Decision tree score:",MLmodel.score(training_features,training_target))

scores = model_selection.cross_val_score(MLalgorithm,poly_features,training_target,scoring='accuracy',cv=50)
print(scores.mean())

## What overfit looks like

![Tree Overfit](tree-overfit.png)

## 4. Evaluate our results and adjust where needed

In [ ]:
MLalgorithm = tree.DecisionTreeClassifier(random_state=1,max_depth=3,min_samples_split=4)
MLmodel = MLalgorithm.fit(training_features,training_target)

print("Decision tree score:",MLmodel.score(training_features,training_target))
scores = model_selection.cross_val_score(MLalgorithm,poly_features,training_target,scoring='accuracy',cv=50)
print(scores.mean())


## Much Different model (and improved result). 
![Tree Pruned](tree-pruned.png)

## 5. Present the model and reiterate (good models only get better).
